In [1]:
# Używane biblioteki
import carla
import random
import cv2
import math
import numpy as np
import csv
import time

In [2]:
#połączenie z symulacją 
client = carla.Client('localhost',2000)
world = client.get_world()

In [ ]:
# Wybór mapy
client.set_timeout(60)
client.load_world('Town05')

In [3]:
#Wczytanie biblioteki z bluprintami pojazdów
bib = world.get_blueprint_library()

In [4]:
#Znalezienie punktów w których można tworzyć pojazdy
spawn_points = world.get_map().get_spawn_points()

In [ ]:
#Stworzenie pojazdu 
pojazd_bib = bib.find('vehicle.lincoln.mkz_2020')
miejsce = spawn_points[8]
miejsce.rotation.yaw = -90

pojazd = world.try_spawn_actor(pojazd_bib, miejsce)
print(miejsce)

In [6]:
# Uzyskanie danych fizycznych pojazdu
fizyka = pojazd.get_physics_control()
#Otrzymanie danych o kołach pojazdu
kola = fizyka.wheels
# Wyznacznie odlegości kół od środka masy pojazdu
lf = abs(miejsce.location.y - 0.01*kola[1].position.y) 
lr = abs(miejsce.location.y - 0.01*kola[3].position.y) 


In [7]:
#Pojazdy
#'vehicle.lincoln.mkz_2020'
#'vehicle.mercedes.coupe_2020'
#'vehicle.mercedes.sprinter'
#'vehicle.ford.ambulance'
#'vehicle.volkswagen.t2_2021'
#'vehicle.mitsubishi.fusorosa'
#'vehicle.tesla.cybertruck'
#'vehicle.dodge.charger_2020'
#'vehicle.nissan.patrol_2021'
#'vehicle.ford.mustang'
#'vehicle.mini.cooper_s_2021'
#'vehicle.nissan.micra'
#'vehicle.tesla.model3'


In [8]:
#Przeniesienie do lokazji utworzonego pojazdu
spectator = world.get_spectator()
transform = carla.Transform(pojazd.get_transform().transform(carla.Location(x=-4,z=2.5)),pojazd.get_transform().rotation) 
spectator.set_transform(transform) 

In [9]:
# Dodanie kamery
pozycja_z = 4 
pozycja_x = -10 

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') 
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=pozycja_z,x=pozycja_x))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=pojazd)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}

camera.listen(lambda image: camera_callback(image,camera_data))

In [11]:

def przyspiesz(predkosc, ustalona_predkosc):
    if predkosc >= ustalona_predkosc:
        return 0
    elif predkosc < ustalona_predkosc:
        return 1
   

In [12]:
#Stworzenie pliku CSV
csv_filename = 'predkosci.csv'

# Nagłówki dla pliku CSV
csv_headers = ['Czas', 'Predkosc (km/h)','Pitch','throttle','brake','Bf','Br']

# Otwórz plik CSV w trybie zapisu i zapisz nagłówki
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_headers)

In [13]:
#Główny program
flaga = 0 # flaga na 0 przyspieszamy do zadanej prędkości flaga na 1 hamujemy
skret = 0
hamulec = 0
pedalGazu = 1
font = cv2.FONT_HERSHEY_SIMPLEX
org2 = (30, 50)
fontScale = 0.5
color = (255, 255, 255)
grubosc = 1
Bf = 0
Br = 0
ustalona_predkosc = 100

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])
status = 0

# Telemetria
#pojazd.show_debug_telemetry()

while True:
  
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        break
    image = camera_data['image']
    v = pojazd.get_velocity() 
   

    s = 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)
    rotacja = pojazd.get_transform().rotation
    yaw = rotacja.yaw
    pitch = rotacja.pitch
    
    current_time = time.time()
    Bf = 0 
    Br = 0
    if status == 1 and abs(v.y) >0.0001:
        dYaw = yaw - yawPoprzednie
        dt = current_time - czasPoprzedni
        YawP = dYaw/dt 

        Bf = math.atan((v.x + lf*YawP)/(v.y)) - skret 

        Br = math.atan((v.x - lr*YawP)/(v.y))        
        

    yawPoprzednie = yaw
    czasPoprzedni = current_time
    status = 1
    # Zapisz do pliku CSV
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([current_time, s ,pitch,pedalGazu,hamulec,Bf,Br])
    image = cv2.putText(image,'Predkosc: '+str(int(s))+' km/h',org2, 
                        font, fontScale,color,grubosc,cv2.LINE_AA)
    
    if flaga==0:
        gaz = przyspiesz(s,ustalona_predkosc)
        pojazd.apply_control(carla.VehicleControl(throttle=gaz, steer=skret))
        
    if s >= ustalona_predkosc:
        flaga = 1
        
    if flaga==1:
        pojazd.apply_control(carla.VehicleControl(throttle=0, brake=1))
        hamulec = 1
        pedalGazu = 0
        
    cv2.imshow('RGB Camera',image)



cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()